In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import max
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import *
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
# Creating the SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Case-Study-Data-Processing") \
    .getOrCreate()

In [3]:
# Read the Csv file

fact_data = spark.read.format("csv").option("header", "true").load("C:\\BigData\\use-case-data-processing-main\\fact.csv")
lookup_data = spark.read.format("csv").option("header", "true").load("C:\\BigData\\use-case-data-processing-main\\lookup.csv")

In [4]:
# Doing BroadCast Join As One Big Data Set and One Small DatSet is present

join_data = fact_data.join(broadcast(lookup_data), fact_data.WEB_PAGEID == lookup_data.WEB_PAGEID, "inner").drop(lookup_data.WEB_PAGEID)
join_data.show(5,False)
join_data.printSchema()
join_data.count()

+---------+----------------+----------+------------+
|USER_ID  |VIEW_TIME       |WEB_PAGEID|WEBPAGE_TYPE|
+---------+----------------+----------+------------+
|149977241|10/02/2016 18:54|3740865072|news        |
|142413313|21/01/2016 16:10|3621915402|news        |
|142413313|21/01/2016 16:20|3621915402|news        |
|142413313|21/01/2016 16:00|3621915402|news        |
|112797679|12/04/2016 13:05|2276268225|news        |
+---------+----------------+----------+------------+
only showing top 5 rows

root
 |-- USER_ID: string (nullable = true)
 |-- VIEW_TIME: string (nullable = true)
 |-- WEB_PAGEID: string (nullable = true)
 |-- WEBPAGE_TYPE: string (nullable = true)



1000

In [5]:
# join_df = fact_data.join(lookup_data, fact_data.WEB_PAGEID ==  lookup_data.WEB_PAGEID, "outer").drop(fact_data.WEB_PAGEID) 
# join_df.show(5,truncate=False)
# join_df.count()

In [6]:
# Convering View_TIME format to 'DD/MM/YYYY' in Column(Data_S)

new_data = join_data.withColumn('Date_S',from_unixtime(unix_timestamp(join_data["VIEW_TIME"],'dd/MM/yyyy mm:ss'),'dd/MM/yyyy'))
new_data.show(5,False)
new_data.printSchema()

+---------+----------------+----------+------------+----------+
|USER_ID  |VIEW_TIME       |WEB_PAGEID|WEBPAGE_TYPE|Date_S    |
+---------+----------------+----------+------------+----------+
|149977241|10/02/2016 18:54|3740865072|news        |10/02/2016|
|142413313|21/01/2016 16:10|3621915402|news        |21/01/2016|
|142413313|21/01/2016 16:20|3621915402|news        |21/01/2016|
|142413313|21/01/2016 16:00|3621915402|news        |21/01/2016|
|112797679|12/04/2016 13:05|2276268225|news        |12/04/2016|
+---------+----------------+----------+------------+----------+
only showing top 5 rows

root
 |-- USER_ID: string (nullable = true)
 |-- VIEW_TIME: string (nullable = true)
 |-- WEB_PAGEID: string (nullable = true)
 |-- WEBPAGE_TYPE: string (nullable = true)
 |-- Date_S: string (nullable = true)



In [7]:
# Convering Data_S from String datatype to Date format in Date field.

data = new_data.withColumn('Date_S',from_unixtime(unix_timestamp(new_data["VIEW_TIME"],'dd/MM/yyyy mm:ss'),'dd/MM/yyyy')) \
.select(col("USER_ID"),("VIEW_TIME"),("WEB_PAGEID"),("WEBPAGE_TYPE"),to_date(col("Date_S"),'dd/MM/yyyy').alias("Date")) 

data.show(5,False)
data.printSchema()

+---------+----------------+----------+------------+----------+
|USER_ID  |VIEW_TIME       |WEB_PAGEID|WEBPAGE_TYPE|Date      |
+---------+----------------+----------+------------+----------+
|149977241|10/02/2016 18:54|3740865072|news        |2016-02-10|
|142413313|21/01/2016 16:10|3621915402|news        |2016-01-21|
|142413313|21/01/2016 16:20|3621915402|news        |2016-01-21|
|142413313|21/01/2016 16:00|3621915402|news        |2016-01-21|
|112797679|12/04/2016 13:05|2276268225|news        |2016-04-12|
+---------+----------------+----------+------------+----------+
only showing top 5 rows

root
 |-- USER_ID: string (nullable = true)
 |-- VIEW_TIME: string (nullable = true)
 |-- WEB_PAGEID: string (nullable = true)
 |-- WEBPAGE_TYPE: string (nullable = true)
 |-- Date: date (nullable = true)



In [8]:
# Calculate difference in days between 2011-12-31 and the Invoice Date

data = data.withColumn("RecencyDays", expr("datediff('2019-10-12', Date)"))

data.show(5,False)

# data.select('USER_ID').distinct().count()

+---------+----------------+----------+------------+----------+-----------+
|USER_ID  |VIEW_TIME       |WEB_PAGEID|WEBPAGE_TYPE|Date      |RecencyDays|
+---------+----------------+----------+------------+----------+-----------+
|149977241|10/02/2016 18:54|3740865072|news        |2016-02-10|1340       |
|142413313|21/01/2016 16:10|3621915402|news        |2016-01-21|1360       |
|142413313|21/01/2016 16:20|3621915402|news        |2016-01-21|1360       |
|142413313|21/01/2016 16:00|3621915402|news        |2016-01-21|1360       |
|112797679|12/04/2016 13:05|2276268225|news        |2016-04-12|1278       |
+---------+----------------+----------+------------+----------+-----------+
only showing top 5 rows



In [9]:
rfm_table = data.groupBy("USER_ID")\
                        .agg(min("RecencyDays").alias("Recency"), \
                             count("WEB_PAGEID").alias("Frequency"))

rfm_table.show()

+----------+-------+---------+
|   USER_ID|Recency|Frequency|
+----------+-------+---------+
|  98343030|    932|        3|
|  59736608|    914|        2|
| 115056661|   1336|        1|
|  88602347|   1259|        1|
| 162690170|   1263|        1|
| 169469168|   1263|        1|
| 167725829|   1280|        1|
| 264289612|   1009|        2|
| 191712149|   1263|        1|
|-247361411|    705|        3|
|  66612057|   1263|        1|
|    788539|   1170|        1|
|   8856308|   1282|        1|
| 171211230|   1262|        1|
| 159147772|   1308|        2|
| 161303572|   1263|        1|
|  78741118|   1349|        2|
|  55620801|   1313|        1|
| 159348322|   1262|        1|
|  96024291|   1361|        1|
+----------+-------+---------+
only showing top 20 rows



In [10]:
# Create a TempView:

data.createOrReplaceTempView("records")

# NEWS PAGE_TYPE

In [11]:
# pageview_news_rec_365

pageview_news_rec_365 = spark.sql("select count(WEB_PAGEID) as pageview_news_rec_365 from records where WEBPAGE_TYPE = 'news' and RecencyDays < '365'")
pageview_news_rec_365.show()

+---------------------+
|pageview_news_rec_365|
+---------------------+
|                   65|
+---------------------+



In [12]:
# pageview_news_rec_365

news_365 = spark.sql("select USER_ID,count(*) as pageview_news_rec_365 from records where WEBPAGE_TYPE = 'news' and RecencyDays < '365' group by USER_ID")
news_365.show(11)
news_365.count()

+----------+---------------------+
|   USER_ID|pageview_news_rec_365|
+----------+---------------------+
| -69271739|                    1|
|-133621877|                    1|
| 157684212|                    1|
|-285299518|                    1|
|-277209141|                    3|
|-151394098|                    3|
| 250009214|                    1|
| -73385561|                    1|
|  78585170|                    1|
|-163187677|                    3|
|-167528374|                   49|
+----------+---------------------+



11

In [13]:
# pageview_news_rec_730

pageview_news_rec_365 = spark.sql("select count(WEB_PAGEID) as pageview_news_rec_730 from records where WEBPAGE_TYPE = 'news' and RecencyDays > '365' and RecencyDays < '730'")
pageview_news_rec_365.show()

+---------------------+
|pageview_news_rec_730|
+---------------------+
|                   35|
+---------------------+



In [14]:
news_730 = spark.sql("select USER_ID,count(*) as pageview_news_rec_730 from records where WEBPAGE_TYPE = 'news' and RecencyDays > '365' and RecencyDays < '730' group by USER_ID")
news_730.show(500)
news_730.count()

+----------+---------------------+
|   USER_ID|pageview_news_rec_730|
+----------+---------------------+
|-247361411|                    3|
|-231838278|                    1|
|-252290668|                    2|
| 155111993|                    1|
|  24160260|                    2|
|-166177270|                    2|
|   9007149|                    1|
|  80169642|                    1|
|-263669125|                    1|
|  61732038|                    4|
|-251309617|                    1|
| 253384086|                    1|
|   1870160|                    1|
| 273780833|                    2|
|  48489723|                    2|
|    951885|                    1|
|   5864422|                    1|
|-244059746|                    1|
|-253796619|                    1|
| 208233177|                    1|
|-216136513|                    2|
| 166709198|                    1|
| 104684497|                    1|
|-175376335|                    1|
+----------+---------------------+



24

In [15]:
# Join news_365 vs news_730
join_df1 = news_730.join(news_365, news_730.USER_ID == news_365.USER_ID, "right").drop(news_730.USER_ID) 
join_df1.show(5,truncate=False)
join_df1.count()

+---------------------+----------+---------------------+
|pageview_news_rec_730|USER_ID   |pageview_news_rec_365|
+---------------------+----------+---------------------+
|null                 |-69271739 |1                    |
|null                 |-133621877|1                    |
|null                 |157684212 |1                    |
|null                 |-285299518|1                    |
|null                 |-277209141|3                    |
+---------------------+----------+---------------------+
only showing top 5 rows



11

In [16]:
# pageview_news_rec_1460

pageview_news_rec_1460 = spark.sql("select count(WEB_PAGEID) as pageview_news_rec_1460 from records where WEBPAGE_TYPE = 'news' and RecencyDays > '730' and RecencyDays < '1460'")
pageview_news_rec_1460.show()

+----------------------+
|pageview_news_rec_1460|
+----------------------+
|                   874|
+----------------------+



In [17]:
news_1460 = spark.sql("select USER_ID,count(*) as pageview_news_rec_1460 from records where WEBPAGE_TYPE = 'news' and RecencyDays > '730' and RecencyDays < '1460' group by USER_ID")
news_1460.show(5)
news_1460.count()

+---------+----------------------+
|  USER_ID|pageview_news_rec_1460|
+---------+----------------------+
| 98343030|                     3|
| 59736608|                     2|
|115056661|                     1|
|162690170|                     1|
|169469168|                     1|
+---------+----------------------+
only showing top 5 rows



746

In [18]:
# pageview_news_rec_2920

pageview_news_rec_2920 = spark.sql("select count(WEB_PAGEID) as pageview_news_rec_2920 from records where WEBPAGE_TYPE = 'news' and RecencyDays > '1460' and RecencyDays < '2920'")
pageview_news_rec_2920.show()

+----------------------+
|pageview_news_rec_2920|
+----------------------+
|                     0|
+----------------------+



In [19]:
news_2920 = spark.sql("select USER_ID,count(*) as pageview_news_rec_2920 from records where WEBPAGE_TYPE = 'news' and RecencyDays > '1460' and RecencyDays < '2920' group by USER_ID")
news_2920.show(5)
news_2920.count()

+-------+----------------------+
|USER_ID|pageview_news_rec_2920|
+-------+----------------------+
+-------+----------------------+



0

In [20]:
# Join news_1460 vs news_2920

join_df2 = news_2920.join(news_1460, news_1460.USER_ID ==  news_2920.USER_ID, "outer").drop(news_1460.USER_ID) 
join_df2.show(5,truncate=False)
join_df2.count()

+----------------------+---------+----------------------+
|pageview_news_rec_2920|USER_ID  |pageview_news_rec_1460|
+----------------------+---------+----------------------+
|null                  |98343030 |3                     |
|null                  |115056661|1                     |
|null                  |59736608 |2                     |
|null                  |162690170|1                     |
|null                  |167725829|1                     |
+----------------------+---------+----------------------+
only showing top 5 rows



746

In [21]:
# Join join_df1 vs join_df2

news_df = join_df1.join(join_df2, join_df1.USER_ID ==  join_df2.USER_ID, "outer").drop(join_df2.USER_ID) \
.select("USER_ID","pageview_news_rec_365","pageview_news_rec_730","pageview_news_rec_1460","pageview_news_rec_2920")

news_df.show(5)

+-------+---------------------+---------------------+----------------------+----------------------+
|USER_ID|pageview_news_rec_365|pageview_news_rec_730|pageview_news_rec_1460|pageview_news_rec_2920|
+-------+---------------------+---------------------+----------------------+----------------------+
|   null|                 null|                 null|                     3|                  null|
|   null|                 null|                 null|                     1|                  null|
|   null|                 null|                 null|                     2|                  null|
|   null|                 null|                 null|                     1|                  null|
|   null|                 null|                 null|                     1|                  null|
+-------+---------------------+---------------------+----------------------+----------------------+
only showing top 5 rows



# MOVIES PAGE_TYPE

In [22]:
# pageview_movies_rec_365

# data.createOrReplaceTempView("records")

pageview_movies_rec_365 = spark.sql("select count(WEB_PAGEID) as pageview_movies_rec_365 from records where WEBPAGE_TYPE = 'movies' and RecencyDays < '365'")
pageview_movies_rec_365.show()

+-----------------------+
|pageview_movies_rec_365|
+-----------------------+
|                      0|
+-----------------------+



In [23]:
# pageview_movies_rec_365

movies_365 = spark.sql("select USER_ID,count(*) as pageview_movies_rec_365 from records where WEBPAGE_TYPE = 'movies' and RecencyDays < '365' group by USER_ID")
movies_365.show(11)
movies_365.count()

+-------+-----------------------+
|USER_ID|pageview_movies_rec_365|
+-------+-----------------------+
+-------+-----------------------+



0

In [24]:
# pageview_news_rec_730

pageview_movies_rec_730 = spark.sql("select count(WEB_PAGEID) as pageview_movies_rec_730 from records where WEBPAGE_TYPE = 'movies' and RecencyDays > '365' and RecencyDays < '730'")
pageview_movies_rec_730.show()

+-----------------------+
|pageview_movies_rec_730|
+-----------------------+
|                      0|
+-----------------------+



In [25]:
movies_730 = spark.sql("select USER_ID,count(*) as pageview_movies_rec_730 from records where WEBPAGE_TYPE = 'movies' and RecencyDays > '365' and RecencyDays < '730' group by USER_ID")
movies_730.show(5)
movies_730.count()

+-------+-----------------------+
|USER_ID|pageview_movies_rec_730|
+-------+-----------------------+
+-------+-----------------------+



0

In [26]:
# Join movies_365 vs movies_730

join_df3 = movies_730.join(movies_365, movies_365.USER_ID == movies_730.USER_ID, "right").drop(movies_365.USER_ID) 
join_df3.show(5,truncate=False)
join_df3.count()

+-----------------------+-------+-----------------------+
|pageview_movies_rec_730|USER_ID|pageview_movies_rec_365|
+-----------------------+-------+-----------------------+
+-----------------------+-------+-----------------------+



0

In [27]:
# pageview_movies_rec_1460

pageview_movies_rec_1460 = spark.sql("select count(WEB_PAGEID) as pageview_movies_rec_1460 from records where WEBPAGE_TYPE = 'movies' and RecencyDays > '730' and RecencyDays < '1460'")
pageview_movies_rec_1460.show()

+------------------------+
|pageview_movies_rec_1460|
+------------------------+
|                      26|
+------------------------+



In [28]:
movies_1460 = spark.sql("select USER_ID,count(*) as pageview_movies_rec_1460 from records where WEBPAGE_TYPE = 'movies' and RecencyDays > '730' and RecencyDays < '1460' group by USER_ID")
movies_1460.show(5)
movies_1460.count()

+---------+------------------------+
|  USER_ID|pageview_movies_rec_1460|
+---------+------------------------+
| 88602347|                       1|
| 78741118|                       2|
| 30317072|                       1|
|197002823|                       1|
|115544304|                       1|
+---------+------------------------+
only showing top 5 rows



18

In [29]:
# pageview_movies_rec_2920

pageview_movies_rec_2920 = spark.sql("select count(WEB_PAGEID) as pageview_movies_rec_2920 from records where WEBPAGE_TYPE = 'movies' and RecencyDays > '1460' and RecencyDays < '2920'")
pageview_movies_rec_2920.show()

+------------------------+
|pageview_movies_rec_2920|
+------------------------+
|                       0|
+------------------------+



In [30]:
movies_2920 = spark.sql("select USER_ID,count(*) as pageview_movies_rec_2920 from records where WEBPAGE_TYPE = 'movies' and RecencyDays > '1460' and RecencyDays < '2920' group by USER_ID")
movies_2920.show(5)
movies_2920.count()

+-------+------------------------+
|USER_ID|pageview_movies_rec_2920|
+-------+------------------------+
+-------+------------------------+



0

In [31]:
# Join movies_1460 vs movies_2920

join_df4 = movies_2920.join(movies_1460, movies_1460.USER_ID ==  movies_2920.USER_ID, "outer").drop(movies_1460.USER_ID) 
join_df4.show(5,truncate=False)
join_df4.count()

+------------------------+---------+------------------------+
|pageview_movies_rec_2920|USER_ID  |pageview_movies_rec_1460|
+------------------------+---------+------------------------+
|null                    |88602347 |1                       |
|null                    |78741118 |2                       |
|null                    |30317072 |1                       |
|null                    |197002823|1                       |
|null                    |115544304|1                       |
+------------------------+---------+------------------------+
only showing top 5 rows



18

In [32]:
# Join join_df3 vs join_df4

movie_df = join_df3.join(join_df4, join_df4.USER_ID ==  join_df3.USER_ID, "outer").drop(join_df4.USER_ID) \
.select("USER_ID","pageview_movies_rec_365","pageview_movies_rec_730","pageview_movies_rec_1460","pageview_movies_rec_2920")

movie_df.show(50)

+-------+-----------------------+-----------------------+------------------------+------------------------+
|USER_ID|pageview_movies_rec_365|pageview_movies_rec_730|pageview_movies_rec_1460|pageview_movies_rec_2920|
+-------+-----------------------+-----------------------+------------------------+------------------------+
|   null|                   null|                   null|                       1|                    null|
|   null|                   null|                   null|                       2|                    null|
|   null|                   null|                   null|                       1|                    null|
|   null|                   null|                   null|                       1|                    null|
|   null|                   null|                   null|                       1|                    null|
|   null|                   null|                   null|                       5|                    null|
|   null|                   

In [33]:
# Joining Movie_df vs News_df

result_df = news_df.join(movie_df, movie_df.USER_ID ==  news_df.USER_ID, "outer").drop(movie_df.USER_ID) \
# select("USER_ID","pageview_news_rec_365","pageview_news_rec_730","pageview_news_rec_1460","pageview_news_rec_2920","pageview_movies_rec_365","pageview_movies_rec_730","pageview_movies_rec_1460","pageview_movies_rec_2920")

result_df.show(5,truncate=False)

+----------+---------------------+---------------------+----------------------+----------------------+-----------------------+-----------------------+------------------------+------------------------+
|USER_ID   |pageview_news_rec_365|pageview_news_rec_730|pageview_news_rec_1460|pageview_news_rec_2920|pageview_movies_rec_365|pageview_movies_rec_730|pageview_movies_rec_1460|pageview_movies_rec_2920|
+----------+---------------------+---------------------+----------------------+----------------------+-----------------------+-----------------------+------------------------+------------------------+
|-69271739 |1                    |null                 |null                  |null                  |null                   |null                   |null                    |null                    |
|-133621877|1                    |null                 |null                  |null                  |null                   |null                   |null                    |null                 